# Lab Notebook 5-6

In this notebook, we will apply the kNN and DT algorithm to a larger exoplanet dataset. We will then examine the effectiveness of the kNN model via a variety of metrics and diagnostics.

# Part 1: Explore data

Start by importing the same modules as in labs 3 and 4, set the matplotlib.rc for graphing, and read in **phl_exoplanet_catalog.csv** using pandas. Additionally, with pandas, use set_option to make sure that the maximum number of columns and rows displayed for our dataframes is 100. Also using set_option, set max_colwidth=100, which modifies the default column width.

In [32]:
import pandas as pd

Find a way to print only the column names of the dataframe. These are the data's features.

In [33]:
df = pd.read_csv("./phl_exoplanet_catalog.csv")

We want to start familiarizing ourselves with our data. Use "describe()" on the dataframe to see some summary statistics. Note down what each of these statistics mean.

In [34]:
df.describe()

,P_STATUS,P_MASS,P_MASS_ERROR_MIN,P_MASS_ERROR_MAX,P_RADIUS,P_RADIUS_ERROR_MIN,P_RADIUS_ERROR_MAX,P_YEAR,P_PERIOD,P_PERIOD_ERROR_MIN,...,S_SNOW_LINE,S_ABIO_ZONE,S_TIDAL_LOCK,P_HABZONE_OPT,P_HABZONE_CON,P_HABITABLE,P_ESI,P_RADIUS_EST,P_MASS_EST,P_SEMI_MAJOR_AXIS_EST
count,4048.0,1598.000000,1467.000000,1467.000000,3139.000000,3105.000000,3105.000000,4048.000000,3.938000e+03,3.807000e+03,...,3786.000000,3.083000e+03,3281.000000,4048.000000,4048.000000,4048.000000,3721.000000,4048.000000,4048.000000,3978.000000
mean,3.0,798.384920,-152.292232,190.289692,4.191426,-0.483990,0.621867,2014.212945,2.309342e+03,-1.073631e+03,...,3.513348,1.768991e+35,0.440103,0.049654,0.034091,0.021986,0.261252,5.588647,323.089993,4.011385
std,0.0,1406.808654,783.366353,1082.061976,4.776830,1.409048,2.007592,3.704839,1.167012e+05,5.943181e+04,...,5.463171,6.944274e+36,0.074285,0.217256,0.181485,0.195731,0.131333,5.392733,965.084290,62.389968
min,3.0,0.019070,-24965.390000,0.000000,0.336300,-54.592700,0.000000,1989.000000,9.070629e-02,-3.650000e+06,...,0.002405,7.293660e-05,0.030707,0.000000,0.000000,0.000000,0.006768,0.000000,0.000000,0.004400
25%,3.0,26.548968,-79.457001,4.449592,1.569400,-0.526870,0.145730,2014.000000,4.497336e+00,-1.129000e-03,...,1.740762,5.264169e-01,0.420337,0.000000,0.000000,0.000000,0.196982,1.703920,3.628101,0.053000
50%,3.0,273.332080,-24.154928,25.108412,2.331680,-0.235410,0.325090,2016.000000,1.187053e+01,-9.392000e-05,...,2.568600,1.429118e+00,0.448357,0.000000,0.000000,0.000000,0.271192,2.667980,7.815324,0.102199
75%,3.0,806.488560,-4.392383,85.813561,3.553570,-0.134520,0.661390,2016.000000,4.186661e+01,-1.594000e-05,...,3.661581,2.641037e+00,0.472140,0.000000,0.000000,0.000000,0.303346,11.770500,149.379160,0.260507
max,3.0,17668.059000,0.270000,26630.808000,77.349000,0.450000,68.919080,2019.000000,7.300000e+06,3.200000e-02,...,104.112780,2.726899e+38,1.322542,1.000000,1.000000,2.000000,0.931208,77.349000,17668.059000,2500.000000


We can group statistics by class. For each possible value of "P_HABITABLE" (0 = not habitable, 1 = possibly habitable, or 2 = probably habitable), display the count for each of the features.

In [35]:
grouped_data = df.groupby("P_HABITABLE")
grouped_data.describe()

P_STATUS                                     P_MASS              \
               count mean  std  min  25%  50%  75%  max   count        mean   
P_HABITABLE                                                                   
0             3993.0  3.0  0.0  3.0  3.0  3.0  3.0  3.0  1575.0  809.993111   
1               21.0  3.0  0.0  3.0  3.0  3.0  3.0  3.0    16.0    1.941373   
2               34.0  3.0  0.0  3.0  3.0  3.0  3.0  3.0     7.0    6.984497   

             ...  P_MASS_EST               P_SEMI_MAJOR_AXIS_EST            \
             ...         75%           max                 count      mean   
P_HABITABLE  ...                                                             
0            ...  156.689210  17668.059000                3923.0  4.063305   
1            ...    2.545899      3.931532                  21.0  0.169579   
2            ...    6.584072      8.921432                  34.0  0.393622   

                                                                            
                   std      min       25%       50%       75%          max  
P_HABITABLE                                                                 
0            62.824346  0.00440  0.052847  0.101407  0.259030  2500.000000  
1             0.197510  0.02144  0.037100  0.089000  0.213000     0.718000  
2             0.280729  0.09100  0.175235  0.258808  0.589277     1.190229  

[3 rows x 776 columns]

# Part 2: Modify data

## Step 2.1

We want a binary classification problem, so lump together "probably" (P_habitable=2) and "possibly" (P_habitable=1) habitable planets in a new dataframe. Check that the two classes are lumped together correctly.

In [36]:
from copy import copy

lumped_df = copy(df)
lumped_df['P_HABITABLE'] = lumped_df['P_HABITABLE'].replace(2, 1)

# Check that the two classes are lumped together correctly
lumped_df['P_HABITABLE'].value_counts()

P_HABITABLE
0    3993
1      55
Name: count, dtype: int64

In [37]:
df['P_HABITABLE'].value_counts()

P_HABITABLE
0    3993
2      34
1      21
Name: count, dtype: int64

## Step 2.2

Let's simplify our data by only using the features 'S_MASS', 'P_PERIOD', and 'P_DISTANCE'. From the dataset created in step 2.1 create a new dataframe called "final_features" that is comprised of the columns 'S_MASS', 'P_PERIOD', and 'P_DISTANCE'. Display the first few rows.

In [38]:
final_features = lumped_df[['S_MASS','P_PERIOD','P_DISTANCE']]

Each column of a data frame is called a *series*. Create a new series named "targets" that is the column "P_HABITABLE". Display the first few rows.

In [39]:
targets = lumped_df['P_HABITABLE']

## Step 2.3

We need to delete data points that contain missing (NaN) values. We can see that NaN values exist by comparing the shape of "final_features" with the count of non-NaN values (using "describe"). Complete these two steps below:

In [42]:
targets.describe()

4048.0

Count the number of NaN values in each column of "final_features" (hint: use isnull()). how many are there in each column?

In [51]:
final_features.isnull().sum()

S_MASS        765
P_PERIOD      110
P_DISTANCE     70
dtype: int64

Remove rows that have one or more NaN values (hint: use "dropna"):

In [52]:
final_features.dropna()

,S_MASS,P_PERIOD,P_DISTANCE
0,2.70,326.030000,1.324418
1,2.78,516.219970,1.534896
2,2.20,185.840000,0.830000
3,0.90,1773.400000,3.130558
4,1.08,798.500000,2.043792
...,...,...,...
4043,0.41,28.165600,0.134560
4044,0.41,7.906961,0.057690
4045,0.12,3.204000,0.021000
4046,0.12,6.689000,0.035000


## Step 2.4

We will now search for outliers in the data and remove them. One quick way to check if there are any is to inspect the distribution of values in each column by creating a histogram. Do this for all three columns:

Due to the wide range of the x-axis (without specifying the range), we can infer that there are outliers. 

We can also tell that there are outliers when we look at the difference between the mean and median for each of the features. Do this below using "describe()".

Time to remove the outliers. 

With scipy.stats.zscore, you can compute the z-score of "final_features". The z-score is the distance between the observed data point and the population mean, scaled by the standard deviation. Keep all data with absolute value of the z-score less than 5 in the array final_features, and remove any data that does not fit this criteria.

Make sure that the data kept and removed in "targets" reflects this change.

Use "final_features.head(n=10)" to show the first 10 rows of "final_features".

As you should see above in "final_features" (and "targets"), the label for each row is not the true row number. In other words, the row label doesn't increase as 0,1,2,3,...

Reset the index of the data frame using "reset_index(drop=True)". This resets the index of the DataFrame, and inserts index into the dataframe columns.

Do the same for "targets".

# Part 3: Explore data (again)

## Step 3.1

We want to check the balance of the data set. Meaning, in "targets", how many ones versus zeros are there?

Print the sum of "targets" (sum of all the ones) divided by the length of "targets".

Also, try using "bincount" on "targets" to show the distribution of ones and zeros:

Now we know that the data set is very imbalanced (many more zeros than ones). This means that we need to be careful when constructing our machine learning model; briefly explain why this is the case.

## Answer:



## Step 3.2

Concatenate "final_features" and "targets" without outliers:

Group the data by "P_HABITABLE", display make one row for P_HABITABLE=0 and another row for P_HABITABLE=1 and use the .describe() method to display summary statistics.

## Step 3.3

Plot the period of orbit as a function of the mass of the parent star. It should look like fig 3.1 in our textbook.

Make sure your graph:
- has a legend (habitable versus not habitable)
- is a scatter plot
- has data (habitable versus not habitable) differentiated by colour
- has a log y-scale
- includes axis labels

# Part 4: Classification

## Step 4.1

Implement train_test_split features and targets. Fix the random state to 3, You can use the default test size, which is 25%. This process will give you Xtrain, Xtest, ytrain, and ytest. Print the shapes of Xtrain and Xtest.

Create a machine learning model by calling "KNeighborsClassifier". Remember that for the kNN algorithm, it is important to standardize the data since it relies on the notion of a metric. To this end, you can use the RobustScaler utility from sklearn.preprocessing. It is also recommended to construct a pipeline with the classifier so that the data is automatically scaled before given to the classifier algorithm. You can use Pipeline from sklearn.pipeline for this. The concept is explained in Chapter 3.4.

Let's see how many 1's ("Habitable") are predicted by our model. Count the number of ones in ytest, and then compare it to the number of ones in the y-array predicted from Xtest:

Let's check the performance of the classifier. Compute the accuracy, precision and recall scores using the **metrics** package for the test data. Also compute the performance of the "lazy" classifier that just assumes y=0 throughout. Any comments? What do these results mean for the success of our classifier? Can we improve our results by modifying the number of nearest neighbours in Step 4.1? Why or why not?

## Step 4.2

Now let's plot the result. Use a scatter plot like in Step 3.3. Make sure that the training and testing points are represented by different shapes. Habitable and non-habitable points should be differentiated by colour as before. Label the axes and include a legend.

## Step 4.3
Repeat steps 4.1 and 4.2 with the DecisionTreeClassifier, random_state=42. Any comments on the results?

# Part 5: Cross-validation
Implement cross-validation on both models for the three metrics accuracy, precision, and recall with stratified k-folds, random shuffling, and 10 splits, random_state=10. What is k-fold validation, and what happens when stratification is applied? Report the mean CV score and the standard deviation. 

# Part 6: Confusion Matrix

Construct the confusion matrices for the two classifiers from above. Please write code from scratch to compute the matrix elements, but feel free to check it against the function provided by sklearn.metrics. Use **cross_val_predict** to assemble the predictions of several folds. You can then use **ConfusionMatrixDisplay()** to visualize, or write your own plotting routine.

# Part 7: ROC curve
Construct the ROC curves for both classifiers. Please use **sklearn's metrics.roc_curve()** to compute the points curve and then plot them. Also draw the ROC curve of a useless (i.e. random) classifier. Don't forget axis labels and a legend. Use cross_val_predict to obtain the probabilities of the predictions. Please also report the ROC AUC score for both classifiers as an average over the 10 folds, using **cross_val_score()**.


# Part 8: Learning curves

With the help of the module **learning_curve** from sklearn.model_selection, construct the learning curves for both classifiers as shown in Fig. 3.7.

# Part 9: Should we exist?
Use the kNN and DT classifiers that you trained above to predict the habitability of the earth.